# Segmenting and Clustering Neighborhoods in Toronto

In [67]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\rashwan\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    tbb-2018.0.5               |       he980bc4_0         150 KB
    ------------------------------------------------------------
                                           Total:         150 KB

The following NEW packages will be INSTALLED:

  tbb                pkgs/main/win-64::tbb-2018.0.5-he980bc4_0




tbb-2018.0.5         | 150 KB    |            |   0% 
tbb-2018.0.5         | 150 KB    | #          |  11% 
tbb-2018.0.5         | 150 KB    | ########5  |  85% 
tbb-2018.0.5         | 150 KB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done



The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2019.07=py37_0
  - defaults/win-64::numba==0.44.1=py37hf9181ef_0


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\rashwan\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    certifi-2019.6.16          |           py37_0         148 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         861 KB

The following NEW packages will be INSTALLED:

  alta

using BeautifulSoup to webscrape the wikipedia page and get the table where we have the postal codes, boroughs and neighbourhoods.

In [113]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.'
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")
tables = soup.find_all('table')
pd.read_html(str(tables[0]), flavor='bs4')
toronto_data = pd.read_html(str(tables[0]), flavor='bs4')[0]
#drop rows where we have boroughs "not assigned"
toronto_data.drop(toronto_data.index[toronto_data['Borough'] == 'Not assigned'], inplace = True)


combine rows where we have same postal codes and different neighbourhoods into single row with nighbourhoods combined. 

In [114]:
fin_toronto_data=toronto_data.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
fin_toronto_data.shape

(103, 3)

read the coordinates data of toronto neighbourhoods

In [115]:
geo_data = pd.read_csv('Geospatial_Coordinates.csv') 
#rename the postal code column to match the same naming we have in the fin_toronto_data dataframe to merge them after
geo_data.rename(columns = {'Postal Code' : 'Postcode'}, inplace = True)


In [116]:
#merge both coordinates and fin_toronto_data dataframes to comeup with the final table including coordinates
neighborhoods = pd.merge(fin_toronto_data, geo_data, on="Postcode")


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [117]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Define Foursquare Credentials and Version

In [118]:
CLIENT_ID = '05FPHY0IDWMACDHOL2M3OCIH2GESB3XX0LHWUOIWYAJCCSJ1' # your Foursquare ID
CLIENT_SECRET = 'VKVQKESB3Q5QLZOXYOXAQ32NTSGW5IOOUB3CCPSF51PLKZTK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


Let's create a function to get the venus for all neighbourhoods

In [119]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [120]:
latitudes = neighborhoods['Latitude'] # neighborhood latitude value
longitudes = neighborhoods['Longitude'] # neighborhood longitude value
names = neighborhoods['Neighbourhood'] # neighborhood name
toronto_venues = getNearbyVenues(names, latitudes, longitudes, radius)

In [134]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 272 uniques categories.


Analyze Each Neighborhood

In [139]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# define a list of column names
cols = toronto_onehot.columns.tolist()
cols

# move the column name to the beggining
cols.insert(0, cols.pop(cols.index('Neighborhood')))
cols

#then use .reindex() function to reorder
toronto_onehot = toronto_onehot.reindex(columns= cols)


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [141]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()


First, let's write a function to sort the venues in descending order.

In [142]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [144]:
import numpy as np

In [147]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)



# Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [150]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [165]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods.rename(columns = {'Neighbourhood' : 'Neighborhood'}, inplace = True)
toronto_merged = neighborhoods

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.dropna(inplace=True)


(99, 16)

Finally, let's visualize the resulting clusters

In [166]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters